In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load features and labels from Google Drive
X = np.load('/content/drive/MyDrive/esm_features_35M.npy')
y = np.load('/content/drive/MyDrive/labels_35M.npy')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_seed)

# Reshape the features for LSTM input (e.g., (num_samples, seq_len, feature_dim))
seq_len = 1  # This should match your sequence length if it's different
X_train = X_train.reshape(-1, seq_len, X_train.shape[1])
X_test = X_test.reshape(-1, seq_len, X_test.shape[1])

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class ImprovedLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_rate):
        super(ImprovedLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.bn = nn.BatchNorm1d(hidden_dim * 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=76, kernel_size=(6, 1), padding=(1, 0))
        self.bn1 = nn.BatchNorm2d(76)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.conv2 = nn.Conv2d(in_channels=76, out_channels=111, kernel_size=(4, 1), padding=(1, 0))
        self.bn2 = nn.BatchNorm2d(111)
        self.conv3 = nn.Conv2d(in_channels=111, out_channels=487, kernel_size=(5, 1), padding=(1, 0))
        self.bn3 = nn.BatchNorm2d(487)

        self.flatten_dim = self._get_flatten_dim(input_dim)
        self.fc = nn.Linear(self.flatten_dim, num_classes)
        self.dropout = nn.Dropout(0.5456158649892608)

    def _get_flatten_dim(self, input_dim):
        h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        x = torch.ones(batch_size, 1, input_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = out.unsqueeze(1).unsqueeze(-1)
        out = self.conv1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        return out.size(1)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
        c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = out.unsqueeze(1).unsqueeze(-1)
        out = self.conv1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        out = self.fc(out)
        return F.log_softmax(out, dim=1)

# Initialize the improved LSTM model with the best hyperparameters
input_dim = X_train.shape[2]
hidden_dim = 181
num_layers = 4
dropout_rate = 0.4397133138964481
learning_rate = 0.0003466440190079221
num_classes = 2

model = ImprovedLSTMClassifier(input_dim, hidden_dim, num_layers, num_classes, dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

n_epochs = 50
patience = 5
best_val_acc = 0.0
early_stop_counter = 0

for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for data, target in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{n_epochs}", leave=False):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)

    train_loss /= total
    train_accuracy = correct / total

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc=f"Validating Epoch {epoch+1}/{n_epochs}", leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

    val_loss /= total
    val_accuracy = correct / total

    print(f'Epoch {epoch+1}/{n_epochs}')
    print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    print(f'Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        early_stop_counter = 0
        model_save_path = '/content/drive/MyDrive/best_improved_lstmCNN_model_35M.pth'
        torch.save(model.state_dict(), model_save_path)
        print("  Best model saved to Google Drive!")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

# Load the best model
print("Loading the best model from Google Drive...")
model.load_state_dict(torch.load('/content/drive/MyDrive/best_improved_lstmCNN_model_35M.pth'))

# Evaluate on the test set
model.eval()
correct = 0
total = 0
y_pred_prob = []
y_true = []

print("Evaluating on the test set...")
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Testing", leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        y_pred_prob.extend(output[:, 1].cpu().numpy())
        y_true.extend(target.cpu().numpy())

test_accuracy = correct / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate additional metrics
roc_auc = roc_auc_score(y_true, y_pred_prob)
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)

# Plot Confusion Matrix
y_pred = np.argmax(model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().detach().numpy(), axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()



In [ ]:
import optuna
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load features and labels from Google Drive
X = np.load('/content/drive/MyDrive/esm_features_35M.npy')
y = np.load('/content/drive/MyDrive/labels_35M.npy')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_seed)

# Reshape the features for LSTM input (e.g., (num_samples, seq_len, feature_dim))
seq_len = 1  # This should match your sequence length if it's different
X_train = X_train.reshape(-1, seq_len, X_train.shape[1])
X_test = X_test.reshape(-1, seq_len, X_test.shape[1])

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_int('hidden_dim', 64, 256)
    num_layers = trial.suggest_int('num_layers', 1, 4)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    class ImprovedLSTMClassifier(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_rate):
            super(ImprovedLSTMClassifier, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=True)
            self.bn = nn.BatchNorm1d(hidden_dim * 2)
            self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 1), padding=(1, 0))
            self.bn1 = nn.BatchNorm2d(64)
            self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
            self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 1), padding=(1, 0))
            self.bn2 = nn.BatchNorm2d(128)
            self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 1), padding=(1, 0))
            self.bn3 = nn.BatchNorm2d(256)

            self.flatten_dim = self._get_flatten_dim(input_dim)
            self.fc = nn.Linear(self.flatten_dim, num_classes)
            self.dropout = nn.Dropout(dropout_rate)

        def _get_flatten_dim(self, input_dim):
            h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
            c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
            x = torch.ones(batch_size, 1, input_dim)
            out, _ = self.lstm(x, (h0, c0))
            out = out[:, -1, :]
            out = out.unsqueeze(1).unsqueeze(-1)
            out = self.conv1(out)
            out = self.bn1(out)
            out = F.relu(out)
            out = self.pool(out)
            out = self.conv2(out)
            out = self.bn2(out)
            out = F.relu(out)
            out = self.pool(out)
            out = self.conv3(out)
            out = self.bn3(out)
            out = F.relu(out)
            out = self.pool(out)
            out = out.view(out.size(0), -1)
            return out.size(1)

        def forward(self, x):
            h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
            c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
            out, _ = self.lstm(x, (h0, c0))
            out = out[:, -1, :]
            out = out.unsqueeze(1).unsqueeze(-1)
            out = self.conv1(out)
            out = self.bn1(out)
            out = F.relu(out)
            out = self.pool(out)
            out = self.conv2(out)
            out = self.bn2(out)
            out = F.relu(out)
            out = self.pool(out)
            out = self.conv3(out)
            out = self.bn3(out)
            out = F.relu(out)
            out = self.pool(out)
            out = out.view(out.size(0), -1)
            out = self.dropout(out)
            out = self.fc(out)
            return F.log_softmax(out, dim=1)

    input_dim = X_train.shape[2]
    num_classes = 2

    model = ImprovedLSTMClassifier(input_dim, hidden_dim, num_layers, num_classes, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    n_epochs = 50
    patience = 5
    best_val_acc = 0.0
    early_stop_counter = 0

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        for data, target in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{n_epochs}", leave=False):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

        train_loss /= total
        train_accuracy = correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc=f"Validating Epoch {epoch+1}/{n_epochs}", leave=False):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item() * data.size(0)
                _, predicted = torch.max(output, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)

        val_loss /= total
        val_accuracy = correct / total

        print(f'Epoch {epoch+1}/{n_epochs}')
        print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        print(f'Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            early_stop_counter = 0
            model_save_path = '/content/drive/MyDrive/best_improved_lstmCNN_model_35M.pth'
            torch.save(model.state_dict(), model_save_path)
            print("  Best model saved to Google Drive!")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping triggered")
                break

    return best_val_acc  # Return validation accuracy for optimization

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Step 4: Print the Best Hyperparameters
print('Best hyperparameters:', study.best_params)
print('Best Accuracy score:', study.best_value)

# Load the best model
print("Loading the best model from Google Drive...")
model.load_state_dict(torch.load('/content/drive/MyDrive/best_improved_lstmCNN_model_35M.pth'))

# Evaluate on the test set
model.eval()
correct = 0
total = 0
y_pred_prob = []
y_true = []

print("Evaluating on the test set...")
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Testing", leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        y_pred_prob.extend(output[:, 1].cpu().numpy())
        y_true.extend(target.cpu().numpy())

test_accuracy = correct / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate additional metrics
roc_auc = roc_auc_score(y_true, y_pred_prob)
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)

# Plot Confusion Matrix
y_pred = np.argmax(model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().detach().numpy(), axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load features and labels from Google Drive
X = np.load('/content/drive/MyDrive/esm_features_35M.npy')
y = np.load('/content/drive/MyDrive/labels_35M.npy')

# Reshape the features for LSTM input (e.g., (num_samples, seq_len, feature_dim))
seq_len = 1  # This should match your sequence length if it's different
X = X.reshape(-1, seq_len, X.shape[1])

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Create TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Initialize parameters
input_dim = X.shape[2]
hidden_dim = 181
num_layers = 4
dropout_rate = 0.4397133138964481
learning_rate = 0.0003466440190079221
num_classes = 2
batch_size = 32
n_epochs = 50
patience = 5

# Initialize KFold
kf = KFold(n_splits=10, shuffle=True, random_state=random_seed)

# Define model class
class ImprovedLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_rate):
        super(ImprovedLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.bn = nn.BatchNorm1d(hidden_dim * 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=76, kernel_size=(6, 1), padding=(1, 0))
        self.bn1 = nn.BatchNorm2d(76)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.conv2 = nn.Conv2d(in_channels=76, out_channels=111, kernel_size=(4, 1), padding=(1, 0))
        self.bn2 = nn.BatchNorm2d(111)
        self.conv3 = nn.Conv2d(in_channels=111, out_channels=487, kernel_size=(5, 1), padding=(1, 0))
        self.bn3 = nn.BatchNorm2d(487)

        self.flatten_dim = self._get_flatten_dim(input_dim)
        self.fc = nn.Linear(self.flatten_dim, num_classes)
        self.dropout = nn.Dropout(0.5456158649892608)

    def _get_flatten_dim(self, input_dim):
        h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        x = torch.ones(batch_size, 1, input_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = out.unsqueeze(1).unsqueeze(-1)
        out = self.conv1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        return out.size(1)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
        c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = out.unsqueeze(1).unsqueeze(-1)
        out = self.conv1(out)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        out = self.pool(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = F.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        out = self.fc(out)
        return F.log_softmax(out, dim=1)

# Cross-validation
fold = 1
accuracies = []
for train_idx, val_idx in kf.split(dataset):
    print(f"Training fold {fold}/{kf.n_splits}")

    # Create data loaders for this fold
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss function, and optimizer
    model = ImprovedLSTMClassifier(input_dim, hidden_dim, num_layers, num_classes, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_acc = 0.0
    early_stop_counter = 0

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        for data, target in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{n_epochs}", leave=False):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

        train_loss /= total
        train_accuracy = correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{n_epochs}", leave=False):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item() * data.size(0)
                _, predicted = torch.max(output, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)

        val_loss /= total
        val_accuracy = correct / total

        print(f'Epoch {epoch+1}/{n_epochs}')
        print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        print(f'Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            early_stop_counter = 0
            model_save_path = f'/content/drive/MyDrive/FinalTest/best_lstmCNN_model_fold_{fold}.pth'
            torch.save(model.state_dict(), model_save_path)
            print(f"  Best model for fold {fold} saved to Google Drive!")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping triggered")
                break

    # Load the best model for this fold
    print(f"Loading the best model for fold {fold} from Google Drive...")
    model.load_state_dict(torch.load(f'/content/drive/MyDrive/FinalTest/best_lstmCNN_model_fold_{fold}.pth'))

    # Evaluate on the validation set
    model.eval()
    correct = 0
    total = 0
    y_pred_prob = []
    y_true = []

    print(f"Evaluating fold {fold} on the validation set...")
    with torch.no_grad():
        for data, target in tqdm(val_loader, desc="Testing", leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            y_pred_prob.extend(output[:, 1].cpu().numpy())
            y_true.extend(target.cpu().numpy())

    val_accuracy = correct / len(val_loader.dataset)
    accuracies.append(val_accuracy)
    print(f'Validation Accuracy for fold {fold}: {val_accuracy:.4f}')
    fold += 1

# Print final cross-validated accuracy
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f'Final Cross-Validated Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}')
